**Завдання**
### Використовуючи з лекції датасет і код:
* створити на основі двошарової нейромережі тришарову;
* оцінити її похибку та точність визначення класів.
#### Після цього використати для оцінки похибки **Mean Squared Error**, в ролі оптимізатора - **Adam**.
* Порівняти, чи змінилась похибка та точність.

In [44]:
# Required libraries.
%matplotlib inline
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision.transforms as transforms
from torchvision import datasets

from tensorflow.keras.datasets import mnist # Біліотека з базою Mnist
from tensorflow.keras.models import Sequential # Підключаємо клас моделі Sequential
from tensorflow.keras.layers import Dense # Підключаємо клас Dense - повнозв'язний шар
from tensorflow.keras.optimizers import Adam # Підключаємо оптимізатор Adam


In [28]:
Transform = transforms.ToTensor()

# Downloading dataset and making a train data.
train_data = datasets.MNIST(root='E:\Python DEVELOPER\StartUpAcademy\projects\lesson_54\DATA',
                            train=False, download=True, transform=Transform)
train_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: E:\Python DEVELOPER\StartUpAcademy\projects\lesson_54\DATA
    Split: Test
    StandardTransform
Transform: ToTensor()

In [29]:
# Making a test data.
test_data = datasets.MNIST(root='E:\Python DEVELOPER\StartUpAcademy\projects\lesson_54\DATA', train=False, download=False, transform=Transform)
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: E:\Python DEVELOPER\StartUpAcademy\projects\lesson_54\DATA
    Split: Test
    StandardTransform
Transform: ToTensor()

In [37]:
# Model's parameters.
input_layer = 784
hidden_layers = [128, 64]
output_layer = 10


In [43]:
# Making a class for neural network.
class Classification(nn.Module):
    # Layers initialisation.
    def __init__(self, input_layer, hidden_layers, output_layer):
        super().__init__() # успадковуємо всі властивості + додаємо своє.
        self.input_layer = nn.Linear(input_layer, hidden_layers[0]) # input layer.
        self.hid_layer1 = nn.Linear(hidden_layers[0], hidden_layers[1]) # hidden layer 1.
        self.hid_layer2 = nn.Linear(hidden_layers[0], hidden_layers[1]) # hidden layer 2
        self.output_layer = nn.Linear(hidden_layers[1], output_layer) # output layer.

    # Path of moving data through the neural network.
    def forward(self, x):
        out = F.relu(self.input_layer(x))  # input layer with a relu activation.
        out = F.relu(self.hid_layer1(out)) # hidden layer 1 with a relu activation.
        out = F.relu(self.hid_layer2(out)) # layer 2 with relu activation.
        out = self.output_layer(out) # output layer.
        return F.log_softmax(out, dim=1) # activation with softmax

# Making an object for the neural network.
model = Classification(input_layer, hidden_layers, output_layer)
model

Classification(
  (input_layer): Linear(in_features=784, out_features=128, bias=True)
  (hid_layer1): Linear(in_features=128, out_features=64, bias=True)
  (hid_layer2): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=10, bias=True)
)

### Loading data into batches:

In [52]:
train_load = DataLoader(train_data, batch_size=100, shuffle=True)
test_load = DataLoader(test_data, batch_size=500, shuffle=False)

### The error function and optimizer:

In [55]:
# Initialisation the error function.
criterion = nn.CrossEntropyLoss()
# Initialization the Adam optimizer.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train the Neural Network:

In [63]:
for images, labels in train_load:
    print('BEFORE [Initial data shape]:', images.size())
    break
print('AFTER [Data after reshaping]:', images.view(100, -1).size())

BEFORE [Initial data shape]: torch.Size([100, 1, 28, 28])
AFTER [Data after reshaping]: torch.Size([100, 784])
